Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic\
для источников paid и email из России - ставим ad\
для источников paid и email не из России - ставим other\
все остальные варианты берем из traffic_source без изменений

In [3]:
import pandas as pd

In [13]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head(1)

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex


In [16]:
visit_log = pd.read_csv('visit_log.csv', sep=';')

# Ввожу функцию, которая будет заполнять строки в source_type
# в соответствии с условиями задачи
def source_type(row):
    
    if row['traffic_source'] in ['yandex', 'google']:
        return 'organic'

    if row['traffic_source'] in ['paid', 'email']:
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'

    return row['traffic_source']

visit_log['source_type'] = visit_log.apply(source_type, axis=1)

visit_log.head(16)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad



Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм\
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [19]:
urls = pd.read_csv('URLs.txt')

news_urls = urls[urls.url.str.contains(r'/\d{8}-')]
news_urls.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [11]:
ratings = pd.read_csv('ratings.csv')

def average_lifetime():   
    ratings = pd.read_csv('ratings.csv')
    lifetime = ratings.groupby('userId').\
        agg({'rating': 'count', 'timestamp': lambda timestamp: max(timestamp) - min(timestamp)}).\
        query('rating > 100')

    lifetime['lifetime'] = pd.to_timedelta(lifetime['timestamp'], unit='s')

    return lifetime.lifetime.mean()
   
average_lifetime()

Timedelta('517 days 17:22:47.734693888')

Задание 4

Дана статистика услуг перевозок клиентов компании по типам 
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов
Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

In [4]:
 
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [12]:
joined = pd.merge(rzd, auto, how='outer', on='client_id')
joined = pd.merge(joined, air, how='outer', on='client_id')
joined.fillna(value=0, inplace=True)
print('Таблица с тремя типами выручки для каждого client_id без указания адреса клиента:')
joined.head(10)

Таблица с тремя типами выручки для каждого client_id без указания адреса клиента:


,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [7]:
joined_adress = joined.merge(client_base, on='client_id')
joined_adress['total_revenue'] = joined_adress.apply(lambda row: sum([value for value in row.values if type(value) == float]), axis=1)
print('Таблица с тремя типами выручки для каждого client_id c указанием адреса клиента:')
joined_adress[['client_id', 'address', 'rzd_revenue', 'auto_revenue', 'air_revenue', 'total_revenue']]

Таблица с тремя типами выручки для каждого client_id c указанием адреса клиента:


,client_id,address,rzd_revenue,auto_revenue,air_revenue,total_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0,1093.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0,2810.0
2,113,Левобережная 1а,10283.0,57483.0,0.0,67766.0
3,114,Мира 14,5774.0,83.0,0.0,5857.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0,1974.0
5,116,Строителей 18,0.0,4834.0,4.0,4838.0
6,117,Панфиловская 33,0.0,98.0,13.0,111.0
7,118,Мастеркова 4,0.0,0.0,173.0,173.0
